In [31]:
import sys
import os 
sys.path.append('../../../Mental_Disorder/3_feature_visualization') # get old tweets library
import age_gender_predictor
from collections import defaultdict
import re
from tabulate import tabulate

# Get user tweets
Read user tweets file & summerize the tweets content to query the age and gender

In [9]:
# Read Patients
def readPatient(folder, filename):
    with open(folder + filename, 'r') as openfile:
#         screen_name<tab>screen_name<tab>tweet_time<tab>tweet_content
        return [line.strip().split('\t') for line in openfile.readlines()]
def checkFolderFile(folder):
    return os.listdir(folder)

In [8]:
print('\n Reading patient tweets..')
folder = '../../twitter crawler/patient_tweets/'
print(folder)
patient_list = checkFolderFile(folder)
patient_tweets_dict = defaultdict(lambda : [])
for patient_name in patient_list:
    patient_tweets_dict[patient_name] = readPatient(folder, patient_name)
    
print(' Patient Number from tweets folder:' + str(len(patient_tweets_dict)))


 Reading patient tweets..
../../twitter crawler/patient_tweets/
 Patient Number from tweets folder:347


## Gender/ Age query function

In [26]:
#Age and gender Distribution
def getAge(timeSeries):
    # This function returns a float, representing the age. 
    texts = ""
    for text in timeSeries:
        texts += text + "\n"
    return age_gender_predictor.get_age(texts)

def getGender(timeSeries):
    # This function returns a float. Positive valuse represents female and vice versa.
    texts = ""
    for text in timeSeries:
        texts += text + "\n"
    return age_gender_predictor.get_gender(texts)


def userValueDistribution(groups, x_name ,  method, legends, colors, bins=50):
    _, plot_axis = plt.subplots(2, 2)
    plot_sequence = ((0,0),(0,1),(1,0),(1,1))
    for g,group in enumerate(groups):
        
        values = [method(timeSeries) for timeSeries in group]
          
        
        plot_axis[plot_sequence[g]].hist(values ,color=colors[g], bins = bins, edgecolor='none' )
        plot_axis[plot_sequence[g]].set_ylabel('User count')
        plot_axis[plot_sequence[g]].set_xlabel(x_name)
        plot_axis[plot_sequence[g]].set_title(str(len(group)) + " " + legends[g] + " people")

        #print("Total People: {} std:".format(len(group)))
        #print("{0} mean :{1:2f} STD:{2:2f}".format(x_name ,np.mean(values),np.std(values)))
    plt.tight_layout()
    plt.show()
    
def getAgeGender(users_tweets_list):
    features = defaultdict(lambda : defaultdict(lambda:None))
    for timeSeries in users_tweets_list:
#         timeSeries = (name, tweets_list)
        features[timeSeries[0]]["age"] = getAge(timeSeries[1])
        features[timeSeries[0]]["gender"] = getGender(timeSeries[1])

    return features


## Organizing file and send to query

In [14]:
# function to delete url
def del_url(line):
    return re.sub(r'(\S*(\.com).*)|(https?:\/\/.*)', "", line)
# replace tag
def checktag(line): 
    return re.sub(r'\@\S*', "", line)

### Prepare query list

In [24]:
# (name, tweets_list)
users_tweets_content_list = []
for user in patient_tweets_dict:
    each_content_list = []
    for tweets in patient_tweets_dict[user]:
#   tweets = screen_name<tab>screen_name<tab>tweet_time<tab>tweet_content
        try:
            each_content_list.append(del_url(checktag(tweets[3])))
        except:
#             print(tweets)
            continue
    users_tweets_content_list.append((user, each_content_list))

### Get Age/ Gender

In [27]:
user_age_gender_dict = getAgeGender(users_tweets_content_list)

In [32]:
headers = ["User Name","Age", "Gender(0 up:female)"]
contents = []
for user in user_age_gender_dict:
    contents.append([user, user_age_gender_dict[user]["age"], user_age_gender_dict[user]["gender"]])

print(tabulate(contents, headers=headers))

User Name                    Age    Gender(0 up:female)
----------------------  --------  ---------------------
ptsdforum                38.0306             0.912824
WeRWorld                 31.9489             0.377751
yobitchmiranda           17.8544             2.30979
quellfoundation          53.4049            -0.316042
sjsmith_au               32.8092            -0.834685
Claudasshian             20.0838             2.12366
silencedrowns            22.2928             2.56852
right2hell               24.7427            -2.03607
PlumBananas              31.7078            -1.58208
reportzoIo               17.5027             2.59505
Erin_Louisee             22.2678             2.37205
thejolson                25.7346            -1.25254
KashawnTaylor            21.0283             0.59909
Music_nerd98             19.276              1.18466
xysuvetexeqi             33.1247            -0.775114
softeyes                 29.0576             2.3448
NeliaCarolyn             23.1485    